In [3]:
def compute_tax(income, brackets):
    """
    Compute tax given taxable income and a list of brackets.
    Each bracket is a tuple: (lower_threshold, upper_threshold, rate).
    """
    tax = 0.0
    for lower, upper, rate in brackets:
        if income > lower:
            taxable = min(income, upper) - lower
            tax += taxable * rate
        else:
            break
    return tax


def effective_salt_cap(
    wages, mortgage_interest, charitable, standard_deduction, TMT, brackets, exemptions
):
    """
    For MFJ:
      - wages: total wage income,
      - mortgage_interest, charitable: other itemized deductions,
      - standard_deduction: MFJ standard deduction,
      - TMT: Tentative Minimum Tax (assumed independent of SALT),
      - brackets: list of tax brackets.

    Assumes the taxpayer itemizes; that is, itemized deduction = SALT + (mortgage_interest + charitable)
    is at least the standard deduction.

    Returns the SALT deduction S* at which the regular tax equals TMT.
    """
    other_deductions = mortgage_interest + charitable
    # For itemizing to occur, we assume:
    # S + other_deductions >= standard_deduction.
    # (For high-income filers, this will hold for any reasonable S.)

    def regular_tax(S):
        # Taxable income = wages - itemized deductions.
        # Itemized deductions = S + (mortgage_interest + charitable).
        TI = wages - (S + other_deductions + exemptions)
        if TI < 0:
            TI = 0
        return compute_tax(TI, brackets)

    # We want to solve f(S) = regular_tax(S) - TMT = 0.
    def f(S):
        return regular_tax(S) - TMT

    # Use a simple binary search for S between 0 and a maximum value.
    # A logical upper bound is wages - other_deductions (which makes TI = 0).
    S_low = 0.0
    S_high = wages - other_deductions  # maximum possible S with nonnegative TI.

    tol = 0.01  # tolerance in dollars
    while S_high - S_low > tol:
        S_mid = (S_low + S_high) / 2
        if f(S_mid) > 0:
            # At S_mid, tax is still above TMT; need to increase S to lower tax.
            S_low = S_mid
        else:
            S_high = S_mid

    return (S_low + S_high) / 2


# --- Parameters for MFJ in 2026 ---
wages = 500_000
mortgage_interest = 15_000
charitable = 10_000
standard_deduction = 16_600
TMT = 115_066
exemptions = 2_748

# 2026 MFJ tax brackets:
# Each bracket is (lower threshold, upper threshold, rate)
brackets = [
    (0, 24_250, 0.10),
    (24_250, 98_500, 0.15),
    (98_500, 198_800, 0.25),
    (198_800, 394_600, 0.28),
    (394_600, 541_000, 0.33),
    (541_000, 611_100, 0.35),
    (611_100, float("inf"), 0.396),
]

S_effective = effective_salt_cap(
    wages, mortgage_interest, charitable, standard_deduction, TMT, brackets, exemptions
)
print("Effective SALT cap: ${:,.2f}".format(S_effective))

Effective SALT cap: $12,183.81
